### Model Preparation

In [1]:
# importing from libraries
import config
import spotipy
import numpy as np
import pandas as pd
import pickle
from sklearn.preprocessing import StandardScaler
from pyjarowinkler import distance
from IPython.display import IFrame
from spotipy.oauth2 import SpotifyClientCredentials

#Initialize SpotiPy with user credentias
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.c_id,
                                                           client_secret= config.c_se))

ModuleNotFoundError: No module named 'config'

In [2]:
# loading top 100 chart (top_chart_df) and song database (upsampled_df)
top_chart_df = pd.read_csv('top_chart_df.csv')
upsampled_df = pd.read_csv('upsampled_df.csv')

NameError: name 'pd' is not defined

In [155]:
# function for loading files with pickle
def load(filename = "filename.pickle"): 
    try: 
        with open(filename, "rb") as f: 
            return pickle.load(f) 
        
    except FileNotFoundError: 
        print("File not found!") 

In [156]:
# loading scaler
scaler = load("Model/scaler.pickle")

In [157]:
# loading kmeans
kmeans = load("Model/kmeans.pickle")

### Function For Song Recommender

In [173]:
# function for recommending similar songs
def song_recommender():
    user_input = input("Please input a song: ")   ### asking user for an input
    song_list_cleaned = [each_string.replace(" ","").lower() for each_string in list(top_chart_df.title)]   ### getting generic song titles by converting to lowercase and removing space
    artist_list_cleaned = [each_string.replace(" ","").lower() for each_string in list(top_chart_df.artist)]   ### getting generic artist names by converting to lowercase and removing space
    user_input_cleaned = user_input.replace(" ", "").lower()   ### getting generic user input of song by converting to lowercase and removing space
    
    duplicate_check = top_chart_df[top_chart_df.title.duplicated(keep=False)]   ### checking for tracks in top chart having the same title
    duplicate_list = [each_string.replace(" ","").lower() for each_string in list(duplicate_check.title)]   ### getting generic duplicate song titles by converting to lowercase and removing space
    
    similarity_count = 0
    similarity_song_list = []
    similarity_score_list = []
    for i in range(0, len(top_chart_df.title)):   ### computing the text similarity between 0 and 1 based on the cleaned user input with each cleaned song title from the top chart
        similarity_score = distance.get_jaro_distance(user_input_cleaned, song_list_cleaned[i], winkler=True, scaling=0.1)
        similarity_song_list.append(top_chart_df.title[i])
        similarity_score_list.append(similarity_score)
        if similarity_score > 0.9:   ### a song is matched with the user input of song if their similarity score is larger than 0.9
            similarity_count = similarity_count+1
        else:
            similarity_count = similarity_count   ### if no song from the top chart has a similarity score more than 0.9, then user input of song is not in top chart and similarity count will be zero
            
    similarity_df = pd.DataFrame({"title":similarity_song_list, "similarity":similarity_score_list})   ### saving the similarity score results in a dataframe
    
    user_input_corrected = similarity_df.loc[similarity_df.similarity == max(similarity_df.similarity), "title"].values[0]   ### correcting the user input of song with the most similar song title from the top chart, given similarity score is larger than 0.9
    user_input_corrected_and_cleaned = user_input_corrected.replace(" ","").lower()   ### getting generic corrected user input of song by converting to lowercase and removing space
    
    ## when there is a song match from the top chart
    if similarity_count>0:
        while True:
            try:
                suggestion = top_chart_df[top_chart_df["title"]!=user_input_corrected].sample(1)   ### looking for another track in the top chart as track suggestion
                suggestion_title = suggestion.iloc[0][0]   ### locating the track suggestion title
                suggestion_artist = suggestion.iloc[0][1]   ### locating the track suggestion artist
                suggestion_id = sp.search(q=suggestion_title+" "+suggestion_artist ,limit=1,market="GB")['tracks']['items'][0]['id']   ### looking for the spotify id of the track suggestion
            except:
                continue
            else:
                break
        
        while True:
            if user_input_corrected_and_cleaned in duplicate_list:   ### checking if there are duplicate titles in the top chart for the user input of song
                print(top_chart_df[top_chart_df.title == user_input_corrected])   ### displaying the duplicate song titles and its artists
                print('\n')
                artist_user_input = input("Which song do you mean? Please input the artist: ")   ### asking user to input the artist
                artist_user_input_cleaned = artist_user_input.replace(" ", "").lower()   ### getting generic user input of song by converting to lowercase and removing space

                suggestion = top_chart_df[(top_chart_df["title"]!=user_input_corrected) | (top_chart_df["artist"]!=artist_user_input)].sample(1)   ### looking for another track in the top chart as track suggestion
                suggestion_title = suggestion.iloc[0][0]   ### locating the track suggestion title
                suggestion_artist = suggestion.iloc[0][1]   ### locating the track suggestion artist
                try:
                    suggestion_id = sp.search(q=suggestion_title+" "+suggestion_artist ,limit=1,market="GB")['tracks']['items'][0]['id']   ### looking for the spotify id of the track suggestion
                    if artist_user_input_cleaned in artist_list_cleaned:   ### when user input of artist is correct
                        break
                except:
                    continue
            else:
                break

        print("\nYour song is in the top 100 chart! Here is another one from the top chart for you.")
        return IFrame(src="https://open.spotify.com/embed/track/"+suggestion_id, width="420", height="80", frameborder="0", allowtransparency="true", allow="encrypted-media",)   ### displaying the embedded spotify player for the recommended song
    
    ## when there is no song match from the top chart
    else:
        print('The song you input is not in the top 100 chart.')
    
        result = sp.search(q=user_input ,limit=5,market="GB")   ### searching with user input of song on spotify and returning top 5 search results
        
        while True:
            if len(result['tracks']['items']) == 0:   ### when there is no result for the search of song
                user_input = input('There is no result for your input, please try again: ')   ### asking user for a new input
                result = sp.search(q=user_input ,limit=5,market="GB")   ### searching with user input of song on spotify and returning top 5 search results
            else:
                for i in range(0,len(result['tracks']['items'])):   ### displaying the top 5 search results
                    print(str(i+1)+". "+(result['tracks']['items'][i]['artists'][0]['name'])+" ("+result['tracks']['items'][i]['name']+")")
                print('\n0. The song I want is not on the list.')
                try:
                    number_input = int(input("Which song and artists do you mean? Please input the number: "))   ### asking user to choose from the top search results
                    print('\n')
                except:
                    print('\nPlease input a vaild number.')
                    continue
                else:
                    if int(number_input) == 0:   ### when the song wanted by user does not show up in the search results
                        user_input = input('Please input a song again, artist name can be included for a better search result: ')   ### asking user for a new input
                        result = sp.search(q=user_input ,limit=5,market="GB")   ### searching with user input of song on spotify and returning top 5 search results
                        continue
                    elif int(number_input) in range(1, len(result['tracks']['items'])+1):   ### when user input a valid number
                        break
                    else:
                        print('Please input a vaild number.')
                        continue
        
        new_input_uri = result['tracks']['items'][number_input-1]['uri']   ### locating the track based on user input
        my_dict = sp.audio_features(new_input_uri)[0]   ### getting the audio features of the chosen track
        my_dict_new = { key: [my_dict[key]] for key in list(my_dict.keys()) }   ### converting the extracted audio features for dataframe format
        audio_feature_df = pd.DataFrame(my_dict_new)   ### saving the extracted audio features in a dataframe

        audio_feature_df.drop(columns=['type','uri','track_href','analysis_url','duration_ms','time_signature','id'],inplace=True)   ### dropping irrelvant columns from audio feature dataframe

        audio_feature_tf_df = scaler.transform(audio_feature_df)   ### scaling the audio feature data with the scaler from the model (refer to Jupyter Notebook 2_Clustering) 
        user_input_cluster = kmeans.predict(audio_feature_tf_df)[0]   ### predicting the cluster of the user in put of song based on the K-means model (refer to Jupyter Notebook 2_Clustering)

        track_id = upsampled_df[upsampled_df['cluster']==user_input_cluster].sample(1)['track_id'].values[0]   ### looking for the track id from the song database, sampling from tracks in the same cluster as the predicted cluster
        print('\n')
        print('Here is another similar song for you.')
        return IFrame(src="https://open.spotify.com/embed/track/"+track_id, width="420", height="80", frameborder="0", allowtransparency="true", allow="encrypted-media",)   ### displaying the embedded spotify player for the recommended song

### Song Recommender

Mechanism: if the song input is on the current Billboard top 100 list, another song from the top chart will be recommended. Otherwise, a similar song will be recommended. Try it out!

In [172]:
# run it and get a song recommendation!
song_recommender()

Please input a song: it feels so good
The song you input is not in the top 100 chart.
1. Sonique (It Feels So Good)
2. Paramore (Misery Business)
3. Sonique (It Feels So Good - 12" Breakbeat Mix)
4. Sonique (It Feels So Good - Radio Edit)
5. Taylor Swift (I Did Something Bad)

0. The song I want is not on the list.
Which song and artists do you mean? Please input the number: 1




Here is another similar song for you.


C:\Users\Bosco\anaconda3\envs\DA_ENV\lib\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
C:\Users\Bosco\AppData\Local\Temp/ipykernel_8308/713919156.py:108: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  track_id = upsampled_df[upsampled_df['cluster']==user_input_cluster][(upsampled_df['genre']==main_genre_1) | (upsampled_df['genre']==main_genre_2)].sample(1)['track_id'].values[0]   ### looking for the track id from the song database, sampling from tracks in the same cluster as the predicted cluster


In [171]:
upsampled_df[upsampled_df['title']=="Astronaut In The Ocean"]

,title,artist,track_id,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,cluster
1853,Astronaut In The Ocean,Masked Wolf,6E90gq0KO6FYZVOXx8kCcC,pop,0.778,0.695,4,-6.865,0,0.0913,0.175,0.0,0.15,0.472,149.996,0


In [81]:
upsampled_df[upsampled_df['cluster']==1].genre.value_counts()

house      137
rnb        137
pop        135
dance      116
hiphop      60
rock        46
techno      44
kpop        43
country     10
jazz         8
metal        2
Name: genre, dtype: int64

In [93]:
upsampled_df[(upsampled_df['genre']==upsampled_df[upsampled_df['cluster']==14].genre.value_counts().index[0]) & (upsampled_df['cluster']==14) ]

,title,artist,track_id,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,cluster
4439,Bass (feat. Tech N9ne & Hopsin),"Merkules, Hopsin, Tech N9ne",24PceGkIgHsyaLXNWDtwCf,hiphop,0.737,0.811,8,-7.123,1,0.207,0.52500,0.048200,0.2090,0.456,142.025,14
4443,What Happened To That Boy,"Baby, Clipse",20Sp7dNLM3NjIu3TInDqjq,hiphop,0.764,0.873,2,-6.210,1,0.291,0.06080,0.000363,0.0757,0.719,93.073,14
4449,Business,Eminem,0x60P5taxdI5pcGbqbap6S,hiphop,0.901,0.813,7,-4.509,1,0.199,0.03250,0.000011,0.1440,0.502,96.442,14
4450,Offbeat,Wais P,39SThuiUXC6L9Yx8RVA7WZ,hiphop,0.575,0.815,6,-5.467,1,0.329,0.05760,0.000000,0.0644,0.779,83.695,14
4451,Wipe the Slate Clean,"BIG HUD, Rittz",3FgN48OjEzszUrIHjOTxDe,hiphop,0.858,0.459,6,-10.312,1,0.269,0.20900,0.000000,0.1040,0.722,134.940,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5216,Just A Lil Bit,50 Cent,4nva9EpKntUTs6CRSGBCn9,hiphop,0.489,0.692,1,-6.672,1,0.410,0.03220,0.006080,0.3150,0.527,96.946,14
5224,Our Way,"Just Juice, Della Kinetic",3wQsGLMB7nOHRvl4Mjh4g9,hiphop,0.775,0.692,8,-6.724,1,0.236,0.18300,0.000000,0.0983,0.274,132.062,14
5228,Ambitionz Az A Ridah,2Pac,5g9lS8deSIxItFBmZRC4vN,hiphop,0.801,0.910,2,-6.566,1,0.225,0.00239,0.000891,0.0737,0.805,87.277,14
5232,Pornographic,Tech N9ne,1HZoGJfy2ds6AVnVnPMUTP,hiphop,0.718,0.796,2,-4.935,1,0.356,0.16500,0.000000,0.1710,0.378,81.924,14


In [128]:
upsampled_df[upsampled_df['cluster']==14][(upsampled_df['genre']==test_genre[0]) | (upsampled_df['genre']==test_genre[1]) | (upsampled_df['genre']==test_genre[2])].sample(1)

C:\Users\Bosco\AppData\Local\Temp/ipykernel_8308/722036926.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  upsampled_df[upsampled_df['cluster']==14][(upsampled_df['genre']==test_genre[0]) | (upsampled_df['genre']==test_genre[1]) | (upsampled_df['genre']==test_genre[2])].sample(1)


,title,artist,track_id,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,cluster
1369,Psycho (feat. Ty Dolla $ign),"Post Malone, Ty Dolla $ign",3swc6WTsr7rl9DqQKQA55C,pop,0.75,0.56,8,-8.094,1,0.105,0.546,0.0,0.111,0.459,140.06,14


In [91]:
upsampled_df[upsampled_df['cluster']==14].genre.value_counts().index[1]

'rnb'

In [125]:
test_genre = upsampled_df[upsampled_df['cluster']==14].genre.value_counts().index[0:3]
test_genre

Index(['country', 'jazz', 'pop'], dtype='object')

In [129]:
upsampled_df[upsampled_df['cluster']==1].genre.value_counts

AttributeError: 'function' object has no attribute 'index'